In [83]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt

In [84]:
# read customers and transactions data

customers = pd.read_csv(r'C:\Users\Nutzer\OneDrive\Desktop\viabill\customers.csv')
transactions = pd.read_csv(r'C:\Users\Nutzer\OneDrive\Desktop\viabill\transactions.csv')

# last available transaction of the client as a credit application
# select the last transaction (transactionID increases with time) for each customer in Transaction table

credit_applications = transactions.groupby(['customerID'], as_index=False).agg({'transactionID':'max'})

# join other related columns of transaction data
new_trans = credit_applications.merge(transactions[['transactionID','price', 'paytmentStatus1',
       'paytmentStatus2', 'paytmentStatus3', 'paytmentStatus4']], on='transactionID', how='left')

# Join transactions df with customers df on customerID
df = pd.merge(new_trans, customers, on='customerID', how='left')

# Replace missing Income values with 999

df['income'] = df['income'].fillna(999)

#create “trans_price_avg_lst3” – for credit applications

df['trans_price_avg_lst3'] = df.groupby('customerID')['price'].rolling(3, min_periods=1).mean().reset_index(0, drop=True)
df['debt_to_income_ratio'] = np.where(df['income']!=999.0,df['price'] / df['income'],999) 
df['income_to_debt_ratio'] = np.where(df['income']!=999.0,df['income'] / df['price'],999) 
df['payment'] = df['price'] / 4
def income_per_age(row):
    if row['age'] <= 20:
        age_group = 1
    elif row['age'] <= 46:
        age_group = 2
    elif row['age'] <= 52:
        age_group = 3
    else:
        age_group = 4
    return row['income'] / age_group

df['income_per_age_group'] = df.apply(income_per_age, axis=1)
df['income_per_age_group'] = np.where(df['income']!=999.0,df['income_per_age_group'],999.0)

df

,customerID,transactionID,price,paytmentStatus1,paytmentStatus2,paytmentStatus3,paytmentStatus4,sex,age,residentialAddress,postalAddress,income,trans_price_avg_lst3,debt_to_income_ratio,income_to_debt_ratio,payment,income_per_age_group
0,1.0,1697842,147,0,0,0,0,1,25,28 Irvine Place,28 Irvine Place,999.0,147.0,999.000000,999.000000,36.75,999.0
1,2.0,1834257,150,0,0,0,0,1,19,72 Bertha Street,72 Bertha Street,43200.0,150.0,0.003472,288.000000,37.50,43200.0
2,3.0,1974624,140,0,0,0,0,2,22,63 Ladberry Street,63 Ladberry Street,70200.0,140.0,0.001994,501.428571,35.00,35100.0
3,4.0,1536291,136,0,1,0,0,0,24,98 Linneman Close,98 Linneman Close,93900.0,136.0,0.001448,690.441176,34.00,46950.0
4,5.0,661121,181,0,0,0,0,1,53,56 Salonica Road,56 Salonica Road,77000.0,181.0,0.002351,425.414365,45.25,19250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490528,499996.0,665043,27,0,0,0,0,1,68,95 Argyle Parade,95 Argyle Parade,40500.0,27.0,0.000667,1500.000000,6.75,10125.0
490529,499997.0,1756533,127,0,0,1,0,2,22,34 Bearke Place,34 Bearke Place,44900.0,127.0,0.002829,353.543307,31.75,22450.0
490530,499998.0,1628579,157,0,0,0,0,1,27,95 Glenhurst Street,95 Glenhurst Street,48700.0,157.0,0.003224,310.191083,39.25,24350.0
490531,499999.0,1722718,171,0,0,0,0,1,55,34 Kiandra Close,34 Kiandra Close,7600.0,171.0,0.022500,44.444444,42.75,1900.0


In [85]:
df.drop(columns=['residentialAddress', 'postalAddress'], axis=1, inplace=True)

In [86]:
df['combined'] = df['paytmentStatus1'].astype(str) + df['paytmentStatus2'].astype(str) + df['paytmentStatus3'].astype(str) + df['paytmentStatus4'].astype(str)

df

,customerID,transactionID,price,paytmentStatus1,paytmentStatus2,paytmentStatus3,paytmentStatus4,sex,age,income,trans_price_avg_lst3,debt_to_income_ratio,income_to_debt_ratio,payment,income_per_age_group,combined
0,1.0,1697842,147,0,0,0,0,1,25,999.0,147.0,999.000000,999.000000,36.75,999.0,0000
1,2.0,1834257,150,0,0,0,0,1,19,43200.0,150.0,0.003472,288.000000,37.50,43200.0,0000
2,3.0,1974624,140,0,0,0,0,2,22,70200.0,140.0,0.001994,501.428571,35.00,35100.0,0000
3,4.0,1536291,136,0,1,0,0,0,24,93900.0,136.0,0.001448,690.441176,34.00,46950.0,0100
4,5.0,661121,181,0,0,0,0,1,53,77000.0,181.0,0.002351,425.414365,45.25,19250.0,0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490528,499996.0,665043,27,0,0,0,0,1,68,40500.0,27.0,0.000667,1500.000000,6.75,10125.0,0000
490529,499997.0,1756533,127,0,0,1,0,2,22,44900.0,127.0,0.002829,353.543307,31.75,22450.0,0010
490530,499998.0,1628579,157,0,0,0,0,1,27,48700.0,157.0,0.003224,310.191083,39.25,24350.0,0000
490531,499999.0,1722718,171,0,0,0,0,1,55,7600.0,171.0,0.022500,44.444444,42.75,1900.0,0000


In [87]:
# Define function to check for default
def check_default(combined):
    if '2' in combined:
        return 2
    elif '1' in combined:
        return 1
    else:
        return 0
# Apply function to create default column
df['default'] = df['combined'].apply(check_default)

df

,customerID,transactionID,price,paytmentStatus1,paytmentStatus2,paytmentStatus3,paytmentStatus4,sex,age,income,trans_price_avg_lst3,debt_to_income_ratio,income_to_debt_ratio,payment,income_per_age_group,combined,default
0,1.0,1697842,147,0,0,0,0,1,25,999.0,147.0,999.000000,999.000000,36.75,999.0,0000,0
1,2.0,1834257,150,0,0,0,0,1,19,43200.0,150.0,0.003472,288.000000,37.50,43200.0,0000,0
2,3.0,1974624,140,0,0,0,0,2,22,70200.0,140.0,0.001994,501.428571,35.00,35100.0,0000,0
3,4.0,1536291,136,0,1,0,0,0,24,93900.0,136.0,0.001448,690.441176,34.00,46950.0,0100,1
4,5.0,661121,181,0,0,0,0,1,53,77000.0,181.0,0.002351,425.414365,45.25,19250.0,0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490528,499996.0,665043,27,0,0,0,0,1,68,40500.0,27.0,0.000667,1500.000000,6.75,10125.0,0000,0
490529,499997.0,1756533,127,0,0,1,0,2,22,44900.0,127.0,0.002829,353.543307,31.75,22450.0,0010,1
490530,499998.0,1628579,157,0,0,0,0,1,27,48700.0,157.0,0.003224,310.191083,39.25,24350.0,0000,0
490531,499999.0,1722718,171,0,0,0,0,1,55,7600.0,171.0,0.022500,44.444444,42.75,1900.0,0000,0


In [88]:
df['default'].value_counts()

0    308366
1    137042
2     45125
Name: default, dtype: int64

In [89]:
df.drop(columns=['paytmentStatus1','paytmentStatus2', 'paytmentStatus3', 'paytmentStatus4','combined'],axis=1,inplace=True)

In [90]:
df

,customerID,transactionID,price,sex,age,income,trans_price_avg_lst3,debt_to_income_ratio,income_to_debt_ratio,payment,income_per_age_group,default
0,1.0,1697842,147,1,25,999.0,147.0,999.000000,999.000000,36.75,999.0,0
1,2.0,1834257,150,1,19,43200.0,150.0,0.003472,288.000000,37.50,43200.0,0
2,3.0,1974624,140,2,22,70200.0,140.0,0.001994,501.428571,35.00,35100.0,0
3,4.0,1536291,136,0,24,93900.0,136.0,0.001448,690.441176,34.00,46950.0,1
4,5.0,661121,181,1,53,77000.0,181.0,0.002351,425.414365,45.25,19250.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
490528,499996.0,665043,27,1,68,40500.0,27.0,0.000667,1500.000000,6.75,10125.0,0
490529,499997.0,1756533,127,2,22,44900.0,127.0,0.002829,353.543307,31.75,22450.0,1
490530,499998.0,1628579,157,1,27,48700.0,157.0,0.003224,310.191083,39.25,24350.0,0
490531,499999.0,1722718,171,1,55,7600.0,171.0,0.022500,44.444444,42.75,1900.0,0


In [91]:
df['customerID'] = df['customerID'].astype(int)
df['customerID'] = df['customerID'].astype(str).str.zfill(7)
df['transactionID'] = df['transactionID'].astype(str)

df

,customerID,transactionID,price,sex,age,income,trans_price_avg_lst3,debt_to_income_ratio,income_to_debt_ratio,payment,income_per_age_group,default
0,0000001,1697842,147,1,25,999.0,147.0,999.000000,999.000000,36.75,999.0,0
1,0000002,1834257,150,1,19,43200.0,150.0,0.003472,288.000000,37.50,43200.0,0
2,0000003,1974624,140,2,22,70200.0,140.0,0.001994,501.428571,35.00,35100.0,0
3,0000004,1536291,136,0,24,93900.0,136.0,0.001448,690.441176,34.00,46950.0,1
4,0000005,661121,181,1,53,77000.0,181.0,0.002351,425.414365,45.25,19250.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
490528,0499996,665043,27,1,68,40500.0,27.0,0.000667,1500.000000,6.75,10125.0,0
490529,0499997,1756533,127,2,22,44900.0,127.0,0.002829,353.543307,31.75,22450.0,1
490530,0499998,1628579,157,1,27,48700.0,157.0,0.003224,310.191083,39.25,24350.0,0
490531,0499999,1722718,171,1,55,7600.0,171.0,0.022500,44.444444,42.75,1900.0,0


#  propose customers segments that we could use when we want to send emails with offers

In [92]:
df.default.value_counts()

0    308366
1    137042
2     45125
Name: default, dtype: int64

In [93]:
segments = df.groupby('default').mean()

segments

C:\Users\Nutzer\AppData\Local\Temp\ipykernel_2380\2576373423.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  segments = df.groupby('default').mean()


,price,sex,age,income,trans_price_avg_lst3,debt_to_income_ratio,income_to_debt_ratio,payment,income_per_age_group
default,,,,,,,,,
0,101.514606,1.154819,44.642499,49046.787681,101.514606,NaN,1102.410769,25.378652,19762.483748
1,101.725785,1.114614,44.629727,49007.581858,101.725785,NaN,1096.700418,25.431446,19690.861072
2,101.861673,1.228565,44.552820,48886.157363,101.861673,NaN,1115.574318,25.465418,19770.557732


In [94]:
df.columns

Index(['customerID', 'transactionID', 'price', 'sex', 'age', 'income',
       'trans_price_avg_lst3', 'debt_to_income_ratio', 'income_to_debt_ratio',
       'payment', 'income_per_age_group', 'default'],
      dtype='object')

# Build model/models that predict if customer will not pay back the loan.

In [98]:
X = df[[ 'trans_price_avg_lst3', 'income_to_debt_ratio',
       'payment', 'income_per_age_group']]
y = df[['default']]

In [99]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
x_scaled = scaler.transform(X)

In [100]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_scaled, y, test_size = 0.3, random_state=42)

In [101]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn import metrics

from sklearn.metrics import accuracy_score, confusion_matrix

from sklearn.metrics import classification_report 


In [102]:
models = [LogisticRegression(),XGBClassifier(),LGBMClassifier(),RandomForestClassifier()]

In [103]:
for i in models:
    models=i
    models.fit(X_train, y_train)
    
    y_pred = models.predict(X_test)
    
    confusion_matrix = metrics.confusion_matrix(y_test, y_pred) 
    
    print('Model Performance for',i)    
   
    print(confusion_matrix)
    print(classification_report(y_test, y_pred))

    print("="*70)
    print("                                      ")

C:\Users\Nutzer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model Performance for LogisticRegression()
[[92347     0     0]
 [41268     0     0]
 [13545     0     0]]


C:\Users\Nutzer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nutzer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nutzer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

              precision    recall  f1-score   support

           0       0.63      1.00      0.77     92347
           1       0.00      0.00      0.00     41268
           2       0.00      0.00      0.00     13545

    accuracy                           0.63    147160
   macro avg       0.21      0.33      0.26    147160
weighted avg       0.39      0.63      0.48    147160

                                      
Model Performance for XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,


C:\Users\Nutzer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Nutzer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model Performance for LGBMClassifier()
[[92337    10     0]
 [41265     3     0]
 [13545     0     0]]


C:\Users\Nutzer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nutzer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nutzer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera

              precision    recall  f1-score   support

           0       0.63      1.00      0.77     92347
           1       0.23      0.00      0.00     41268
           2       0.00      0.00      0.00     13545

    accuracy                           0.63    147160
   macro avg       0.29      0.33      0.26    147160
weighted avg       0.46      0.63      0.48    147160

                                      
Model Performance for RandomForestClassifier()
[[64980 21407  5960]
 [28984  9632  2652]
 [ 9576  3094   875]]
              precision    recall  f1-score   support

           0       0.63      0.70      0.66     92347
           1       0.28      0.23      0.26     41268
           2       0.09      0.06      0.08     13545

    accuracy                           0.51    147160
   macro avg       0.33      0.33      0.33    147160
weighted avg       0.48      0.51      0.49    147160

                                      


### Assuming logreg performed well building model and predicting probabilities

In [104]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [105]:
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
y_prob = logreg.predict_proba(X_test)[:,2]

C:\Users\Nutzer\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [106]:
# Define column names
cols_x = [ 'trans_price_avg_lst3', 'income_to_debt_ratio',
       'payment', 'income_per_age_group']
cols_y = ['default']
# Convert the array into a DataFrame with column names
data_test_x = pd.DataFrame(X_test, columns=cols_x)
data_test_y = pd.DataFrame(y_test, columns=cols_y)


In [107]:
data_test_y = data_test_y.reset_index(drop=True)

data_test_y

,default
0,0
1,1
2,0
3,0
4,0
...,...
147155,0
147156,2
147157,1
147158,1


In [108]:
data_test_x

,trans_price_avg_lst3,income_to_debt_ratio,payment,income_per_age_group
0,0.793685,-0.264072,0.793685,1.051645
1,-0.937249,-0.361443,-0.937249,-0.949506
2,0.041865,-0.422179,0.041865,-0.777919
3,-1.304417,0.709878,-1.304417,-0.084856
4,0.653812,-0.320827,0.653812,0.369435
...,...,...,...,...
147155,-0.307818,-0.150065,-0.307818,0.753953
147156,1.108400,-0.437969,1.108400,-0.962426
147157,0.426517,-0.318716,0.426517,-0.248690
147158,-0.167945,-0.219483,-0.167945,-0.367043


In [109]:
sample = data_test_x.join(data_test_y)
sample

,trans_price_avg_lst3,income_to_debt_ratio,payment,income_per_age_group,default
0,0.793685,-0.264072,0.793685,1.051645,0
1,-0.937249,-0.361443,-0.937249,-0.949506,1
2,0.041865,-0.422179,0.041865,-0.777919,0
3,-1.304417,0.709878,-1.304417,-0.084856,0
4,0.653812,-0.320827,0.653812,0.369435,0
...,...,...,...,...,...
147155,-0.307818,-0.150065,-0.307818,0.753953,0
147156,1.108400,-0.437969,1.108400,-0.962426,2
147157,0.426517,-0.318716,0.426517,-0.248690,1
147158,-0.167945,-0.219483,-0.167945,-0.367043,1


In [110]:
cols_pred = ['default_predicted']
cols_prob = ['default_predicted_probability (%)']
# Convert the array into a DataFrame with column names
data_pred = pd.DataFrame(y_pred, columns=cols_pred)
data_prob = pd.DataFrame(y_prob, columns=cols_prob)

In [111]:
sample = sample.join(data_pred)
sample

,trans_price_avg_lst3,income_to_debt_ratio,payment,income_per_age_group,default,default_predicted
0,0.793685,-0.264072,0.793685,1.051645,0,0
1,-0.937249,-0.361443,-0.937249,-0.949506,1,0
2,0.041865,-0.422179,0.041865,-0.777919,0,0
3,-1.304417,0.709878,-1.304417,-0.084856,0,0
4,0.653812,-0.320827,0.653812,0.369435,0,0
...,...,...,...,...,...,...
147155,-0.307818,-0.150065,-0.307818,0.753953,0,0
147156,1.108400,-0.437969,1.108400,-0.962426,2,0
147157,0.426517,-0.318716,0.426517,-0.248690,1,0
147158,-0.167945,-0.219483,-0.167945,-0.367043,1,0


In [112]:
sample = sample.join(data_prob)
sample

,trans_price_avg_lst3,income_to_debt_ratio,payment,income_per_age_group,default,default_predicted,default_predicted_probability (%)
0,0.793685,-0.264072,0.793685,1.051645,0,0,0.092532
1,-0.937249,-0.361443,-0.937249,-0.949506,1,0,0.090828
2,0.041865,-0.422179,0.041865,-0.777919,0,0,0.091390
3,-1.304417,0.709878,-1.304417,-0.084856,0,0,0.091789
4,0.653812,-0.320827,0.653812,0.369435,0,0,0.092188
...,...,...,...,...,...,...,...
147155,-0.307818,-0.150065,-0.307818,0.753953,0,0,0.091898
147156,1.108400,-0.437969,1.108400,-0.962426,2,0,0.091932
147157,0.426517,-0.318716,0.426517,-0.248690,1,0,0.091864
147158,-0.167945,-0.219483,-0.167945,-0.367043,1,0,0.091569
